In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import textblob
from textblob import TextBlob
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
import re

In [2]:
data = pd.read_csv("stock_data.csv", encoding = "ISO-8859-1", engine='python')

Tr - Twitterdan toplanmış ve 2'ye bölünmüş hisse senedi haberleri.

ENG - Gathered Stock news from Multiple twitter Handles regarding Economic news dividing into two parts : Negative(-1) and positive(1) .

Negative count: 2,106

Positive count: 3,685


In [3]:
data

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1
...,...,...
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off dayâs high, up...",1


Tr - Veri setine ilk bakış

Eng - First look at the dataset

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5791 entries, 0 to 5790
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       5791 non-null   object
 1   Sentiment  5791 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.6+ KB


In [5]:
data["Sentiment"].value_counts()

 1    3685
-1    2106
Name: Sentiment, dtype: int64

Tr - Veri ön işleme

Eng - Data preprocessing

In [6]:
data["Sentiment"] = data["Sentiment"].replace(-1,0)

In [7]:
sw = stopwords.words('english')

In [8]:
#data["Text"] = data["Text"].apply(lambda x: TextBlob(x).words)

In [9]:
tokenizer = Tokenizer()

In [10]:
tokenizer.fit_on_texts(data["Text"])

In [11]:
data.head(5)

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


Tr- Basit bir özellik çıkarımı.
Text verilerimizi tokenleştirip kelimeleri bir integer olarak atıyoruz. Sonra modele vereceğimiz cümlelerimizin boyutunu belirliyoruz. Bu önemlidir, çünkü çok fazla boyutlu kelimeler modelimizi yorabilir veya az geçen, bir öneme sahip olmaya kelimelerimizide vermiş olabiliriz.

Eng- Simple feature extraction.
We specify our text data and throw out the words as whole numbers. Then we determine the size of our sentences that we will give to the model.This is important because we can tire our model of words with too many dimensions, or we may have given meaningless words to the model.


In [12]:
data_tokens = tokenizer.texts_to_sequences(data["Text"])

In [13]:
data_tokens[15]

[742, 1317, 269, 395, 1009, 1010, 4629, 468, 205]

In [14]:
data["Text"][15]

'HA Hitting 35.65 means resume targeting 42 level ..  '

Tr - Buraya kadar tokenleştirme işlemini yaptık. Burdan sonra ise özellik çıkarımı yaparak kelimelerimizi hem vektörleştiriyoruz, hemde dediğimiz gibi modelimize vereceğimiz cümlelerin boyutunu beliyorluyoruz.

Eng- After that, we do feature extraction. We both vectorize and, as I said, determine the sentences we will give to our model.

In [15]:
num_tokens = [len(tokens) for tokens in data_tokens]

In [16]:
num_tokens = np.array(num_tokens)

In [17]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

28

In [18]:
#np.sum(num_tokens < max_tokens) / len(num_tokens)

Tr - Veri setimizde 26'dan fazla kelimeden oluşan cümlelerimizin veri setimize oranı sadece %7.

Eng - The ratio of our sentences consisting of more than 26 words in the data set to our data set is only %7.

In [19]:
data_pad = pad_sequences(data_tokens, maxlen = max_tokens)

In [20]:
data_pad.shape

(5791, 28)

Tr - Model Eğitimi

Eng - Model Training

In [21]:
train_sequences, test_sequences, y_train, y_test = train_test_split(data_pad, data["Sentiment"], train_size=0.8, shuffle=True, random_state=1)

Tr - Modelimizi oluşutuyoruz.

Eng - We create our model.

In [22]:
model = Sequential()

Tr - Kelime matrisimizi oluşturuyoruz. Ben deneyerek en başarılı paramtreleri bu şekilde buldum.

Eng - We create our word matrix. This is how I found the most successful parameters by trying.

In [23]:
model.add(Embedding(input_dim = 50000,
                    output_dim = 300,
                   input_length = max_tokens,
                   name = "embeding_layer"))

In [24]:
model.add(GRU(units = 64, return_sequences = True))
model.add(GRU(units = 32, return_sequences = True))
model.add(GRU(units = 16, return_sequences = True))
model.add(GRU(units = 8, return_sequences = True))
model.add(GRU(units = 4, return_sequences = False))
model.add(Dense(1, activation = 'sigmoid'))

In [25]:
optimizer = Adam(lr = 3e-5)

In [26]:
model.compile(loss = 'binary_crossentropy',
              optimizer = optimizer,
              metrics = ["accuracy"])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embeding_layer (Embedding)   (None, 28, 300)           7500000   
_________________________________________________________________
gru (GRU)                    (None, 28, 64)            70272     
_________________________________________________________________
gru_1 (GRU)                  (None, 28, 32)            9408      
_________________________________________________________________
gru_2 (GRU)                  (None, 28, 16)            2400      
_________________________________________________________________
gru_3 (GRU)                  (None, 28, 8)             624       
_________________________________________________________________
gru_4 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5

In [28]:
model.fit(train_sequences, y_train, epochs = 20, batch_size = 32)

Epoch 1/20
145/145 [==============================] - 33s 228ms/step - loss: 0.6779 - accuracy: 0.6235
Epoch 2/20
145/145 [==============================] - 34s 231ms/step - loss: 0.6569 - accuracy: 0.6332
Epoch 3/20
145/145 [==============================] - 32s 222ms/step - loss: 0.6459 - accuracy: 0.6332
Epoch 4/20
145/145 [==============================] - 36s 251ms/step - loss: 0.6322 - accuracy: 0.6364
Epoch 5/20
145/145 [==============================] - 32s 223ms/step - loss: 0.5996 - accuracy: 0.6734
Epoch 6/20
145/145 [==============================] - 31s 213ms/step - loss: 0.5297 - accuracy: 0.7649
Epoch 7/20
145/145 [==============================] - 31s 216ms/step - loss: 0.4483 - accuracy: 0.8413
Epoch 8/20
145/145 [==============================] - 37s 252ms/step - loss: 0.3856 - accuracy: 0.8772
Epoch 9/20
145/145 [==============================] - 32s 221ms/step - loss: 0.3429 - accuracy: 0.9011
Epoch 10/20
145/145 [==============================] - 32s 220ms/step - l

In [29]:
results = model.evaluate(test_sequences, y_test, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

    Test Loss: 0.54987
Test Accuracy: 76.10%
